In [1]:
#Imports
from scripts import dataset_scripts as ds_s, mauve_quantization as mq, subset_selection as ss
from transformers import AutoTokenizer, AutoModel
import torch
from datasets import Dataset, load_from_disk
import numpy as np

In [2]:
# Dicts are in format: {ds_path, ds_name, under_ds_name}
#human_ds_dict = {"ds_path":"data/human/news-fi-2019.jsonl", "ds_name":"news-fi-2019.jsonl", "under_ds_name":None}
#clums_ds_dict = {"ds_path":"data/clumsified/news-fi-2019.jsonl_regeneration_5_mini_regen_round_1.jsonl", "ds_name":"news-fi-2019.jsonl_regeneration_5_mini_regen_round_1.jsonl", "under_ds_name":"news-fi-2019.jsonl"}


#ds = ds_s.format_datasets([human_ds_dict, clums_ds_dict])
#ref, remaining = ds_s.sample_reference_corpus(ds, "news-fi-2019.jsonl", 5000)

In [3]:
#model_name = "intfloat/multilingual-e5-large-instruct"
import os
os.environ['HF_HOME'] = '/scratch/project_2000539/tapio/HF_cache/'

In [4]:
#Model name that can be loaded from HF goes here
#model_id = model_name
#tokenizer = AutoTokenizer.from_pretrained(model_id)
#model = AutoModel.from_pretrained(model_id, device_map="auto")


# ## Getting the embeddings with the wanted (L)LM

#inputs_q = [tokenizer.encode(x['text'], return_tensors="pt", truncation=True, max_length=512) for x in remaining]
#embeddings_q = mq.featurize_tokens_from_model(model, inputs_q, 1, name="", verbose=False)
#inputs_q = []
#del inputs_q
#inputs_p = [tokenizer.encode(x['text'], return_tensors="pt", truncation=True, max_length=512) for x in ref]
#embeddings_p = mq.featurize_tokens_from_model(model, inputs_p, 1, name="", verbose=False)
#inputs_p = []
#del inputs_p

In [5]:
#for i in range(len(ref)):
#    ref[i]['embedding']= embeddings_p[i]

#for i in range(len(remaining)):
#    remaining[i]['embedding']= embeddings_q[i]


#torch.save(ref, 'data/embs/ref.pt')

#ds_test = Dataset.from_list(ref)
#ds_test.save_to_disk("data/embs/ref.hf")

#ds_test = Dataset.from_list(remaining)
#ds_test.save_to_disk("data/embs/remaining.hf")

#torch.load(remaining, 'data/embs/remaining.pt')

In [6]:
ds = load_from_disk("data/prepped_dataset/news_run_1_ppls.hf").to_list()

In [7]:
ref, remaining = ds_s.sample_reference_corpus(ds, "news-fi-2019.jsonl", 5000)

In [8]:
embeddings_p = [torch.tensor(x['embedding']) for x in ref]
embeddings_q = [torch.tensor(x['embedding']) for x in remaining]

In [9]:
#Estimate the optimal number of clusters as done in MAUVE
num_of_clusters = max(2, int(round(min(len(embeddings_p)/10, len(embeddings_q)/10))))

print(f'The number of clusters is {num_of_clusters}')
results = mq.CDOE(torch.cat(embeddings_p), torch.cat(embeddings_q), num_of_clusters, verbose=True)

The number of clusters is 500
seed = 0
performing clustering in lower dimension = 392
'Shape of embeddings before PCA transformation (43030, 1024)'
'Shape of embeddings after PCA transformation (43030, 393)'
Clustering 43030 points in 393D to 500 clusters, redo 5 times, 500 iterations
  Preprocessing in 0.01 s
Outer iteration 0 / 5
  Iteration 499 (111.15 s, search 107.35 s): objective=1.39086e+07 imbalance=1.841 nsplit=0       
Objective improved: keep new clusters
Outer iteration 1 / 5
  Iteration 499 (220.03 s, search 213.10 s): objective=1.38641e+07 imbalance=1.722 nsplit=0       
Objective improved: keep new clusters
Outer iteration 2 / 5
  Iteration 499 (328.43 s, search 318.35 s): objective=1.39168e+07 imbalance=1.883 nsplit=0       
Outer iteration 3 / 5
  Iteration 499 (446.06 s, search 429.54 s): objective=1.39213e+07 imbalance=1.816 nsplit=0       
Outer iteration 4 / 5
kmeans time: 560.82 s59 s, search 538.95 s): objective=1.39001e+07 imbalance=1.816 nsplit=0       


In [10]:
# Combining information to make future work easier

p2cluster = results['p2cluster']
for i in p2cluster:
    ref[i]['cluster_id'] = int(p2cluster[i])
q2cluster = results['q2cluster']
for i in q2cluster:
    remaining[i]['cluster_id'] = int(q2cluster[i])

In [11]:
def _normalize(array):
        # Normalize sum of array to 1.
        # We assume non-negative entries with non-zero sum.
        return array / array.sum()

In [12]:
from scripts import dataset_scripts as ds_s, mauve_quantization as mq, subset_selection as ss
p_distr = results['p_bin_counts']
q_distr = np.histogram(
        list(q2cluster.values()), bins=num_of_clusters,
        range=[0, num_of_clusters], density=False
    )[0]
#q_distr = results['q_bin_counts']

In [13]:
q_human = [x for x in remaining if not x['under_collection']]
q_clums = [x for x in remaining if x['under_collection']]

In [14]:
test_simple = ss.get_target_n_per_cluster(p_distr, q_distr, 5000, True)
print(np.sum(list(test_simple.values())))

4998


In [15]:
test_complex = ss.get_target_n_per_cluster(p_distr, q_distr, 5000)
print(np.sum(list(test_complex.values())))

5000


In [16]:
# Writing the function to do a random subset that matches the 'target distribution'
import random
def random_subset_sampler(target_distribution, q2cluster):
    cluster2q = {}
    for key, value in q2cluster.items():
        if value in cluster2q:
            cluster2q[value].append(key)
        else:
            cluster2q[value] = [key]

    returnable = []
    for c in target_distribution:
        if target_distribution[c] > 0:
            returnable += random.sample(cluster2q[c], target_distribution[c])
    return returnable

def ppl_ordered_subset_sampler(target_distribution, q2cluster, remaining, order_column):
    cluster2q = {}
    for key, value in q2cluster.items():
        if value in cluster2q:
            cluster2q[value].append(key)
        else:
            cluster2q[value] = [key]

    returnable = []
    for c in target_distribution:
        if target_distribution[c] > 0:
            sorted_embs = sorted(cluster2q[c], key=lambda index: remaining[index][order_column])
            returnable += sorted_embs[:target_distribution[c]]
    return returnable
            
    



In [21]:
test_sample = random_subset_sampler(test_simple, q2cluster)
test_sample_unders = [remaining[i]['under_collection_id'] for i in test_sample]
print(f'Ratio of human texts in subset (using simple per-cluster counts): {len([x for x in test_sample_unders if x])/len(test_sample_unders)}')

Ratio of human texts in subset (using simple per-cluster counts): 0.4943977591036415


In [22]:
test_sample = ppl_ordered_subset_sampler(test_simple, q2cluster, remaining, 'cppl')
test_sample_unders = [remaining[i]['under_collection_id'] for i in test_sample]
print(f'Ratio of human texts in subset when ordered by ppl (using complex per-cluster counts): {len([x for x in test_sample_unders if x])/len(test_sample_unders)}')

Ratio of human texts in subset when ordered by ppl (using complex per-cluster counts): 0.6572629051620649


In [23]:
test_sample = random_subset_sampler(test_complex, q2cluster)
test_sample_unders = [remaining[i]['under_collection_id'] for i in test_sample]
print(f'Ratio of human texts in subset (using simple per-cluster counts): {len([x for x in test_sample_unders if x])/len(test_sample_unders)}')

Ratio of human texts in subset (using simple per-cluster counts): 0.4992


In [24]:
test_sample = ppl_ordered_subset_sampler(test_complex, q2cluster, remaining, 'cppl')
test_sample_unders = [remaining[i]['under_collection_id'] for i in test_sample]
print(f'Ratio of human texts in subset when ordered by ppl (using complex per-cluster counts): {len([x for x in test_sample_unders if x])/len(test_sample_unders)}')

Ratio of human texts in subset when ordered by ppl (using complex per-cluster counts): 0.6572
